In [1]:
import sacct
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import os
import dask.dataframe as dd
from random import randint
import numpy as np
from dask.array import from_array as fa

sourceFile = '5000mb.csv'  # Source of sacct data if using offline mode.

from dask_jobqueue import SLURMCluster
from dask.distributed import Client

#cluster = SLURMCluster() #add arguments if want a cluster outside of the set up defult
cluster = SLURMCluster(cores=24, processes=6, walltime="1:00:00", queue="compute", memory= "120GB")

In [2]:
cluster.scale(jobs=2)

In [3]:
client = Client(cluster)
client

Client Scheduler: tcp://10.22.254.12:38641 Dashboard: http://10.22.254.12:8787/status,Cluster Workers: 12 Cores: 48 Memory: 240.00 GB


In [4]:
df = dd.read_csv(sourceFile)
#df = dd.read_parquet(sourceFile)  # For if you're smart and use parquet files instead of csvs

In [5]:
pd.set_option('display.max.columns', None)
pd.set_option('display.width', 150)
print(df.shape,'\n\n')
print(df.head(),'\n\n')
df.describe()

(Delayed('int-7e69a961-7e77-4f4a-aec6-a88a882b38ca'), 76) 


   Unnamed: 0  JobIDRaw   step           JobID  UID  GID  AssocID  Cluster          JobName  User  Group  Account  Reservation  ReservationId  \
0         NaN  JobIDRaw   step           JobID  UID  GID  AssocID  Cluster          JobName  User  Group  Account  Reservation  ReservationId   
1    177868.0  33929685  batch  33929685.batch  NaN  NaN    14492    comet            batch   NaN    NaN   mit136          NaN            NaN   
2     12583.0  33809406  batch  33809406.batch  NaN  NaN    20768    comet            batch   NaN    NaN   sio124          NaN            NaN   
3      1516.0  33781793     45     33781793.45  NaN  NaN    20917    comet  hydra_pmi_proxy   NaN    NaN   csd606          NaN            NaN   
4    115172.0  33883732  batch  33883732.batch  NaN  NaN    22537    comet            batch   NaN    NaN   uok114          NaN            NaN   

   Partition  QOS  QOSRAW  NNODES  NTASKS  NCPUS  AllocCPUS  ReqCPUS

,Unnamed: 0
npartitions=1,
,float64
,...


In [6]:
# Example of populating empty column with new data

runTimes = df['CPUTimeRaw'].compute().tolist()

limitExamples = []
for time in runTimes:
    newTime = round((time+randint(0,100)/10)*(1+randint(0,100)/100))
    limitExamples.append(newTime)

print(runTimes[:10])
print(limitExamples[:10])
df['Timelimit'] = fa(limitExamples)
print(df['Timelimit'])
dd.to_csv(df,'df_sacctFix.csv')

TypeError: can only concatenate str (not "float") to str

In [ ]:
%%time
# Time wasted query
timeWaste = []
timeEff = []

timeLimit = df['Timelimit'].compute().tolist()
cpuTime = df['CPUTimeRaw'].compute().tolist()

    #timeEff.append(round(100*(value/df['Timelimit'].compute().tolist()[ind])))

In [ ]:
%%time
for ind, value in enumerate(cpuTime):
    timeWaste.append(timeLimit[ind] - value)

In [ ]:
print(cpuTime[:10])
print(timeLimit[:10])
print((timeWaste)[:10])

In [ ]:
# Load in an old query rather than build a new one
queryDF = pd.read_parquet('sacctQuery.parquet')

In [ ]:
# Construct dictionary and pandas df with old and new data to display query
#newCatDir = {'CPUTimeRaw':df['CPUTimeRaw'].compute().tolist(), 'TimeLimit':df['Timelimit'].compute().tolist(), 'TimeWaste': timeWaste}
newCatDir = {'JobIDRaw': df['JobIDRaw'],'CPUTimeRaw': df['CPUTimeRaw'], 'TimeLimit': df['Timelimit'], 'TimeWaste': timeWaste}#, 'TimeEff%': timeEff}

queryDF = pd.DataFrame(newCatDir)

In [ ]:
# Display query dataframe preview
queryDF.head(50)

In [ ]:
queryDF.describe()

In [ ]:
# Sorting query df
queryDF.sort_values('TimeWaste', ascending=False)

In [ ]:
# Query save system: If desired, users can save their current query as a parquet file for use later

saveName = 'sacctQuery'  # Doesn't need to include the .parqet or .csv extension
queryDF.to_parquet(saveName+'.parquet')
